# CS109b Final Project

## Milestone2

by Danqing Wang, Wenshan Zheng, Zecai Liang

----------

In [3]:
#### load libraries

import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

---
## Part1. Download Data

For demenstration of code:
use the the top 100 rows (the top 100 movies sorted by "popularity" in TMDB database)

### 1.1 Download Data from TMDB

by Wenshan

input:

n = 25k (number of data points we decide to download)

output: 


data file "imdb_id25K.txt"

data file "data_imdb_top100.txt"

### 1.2 Download Data from IMDB (DONE)

by Danqing

input: 

imdb_id25K.txt



output: 

data file "data_imdb_top100.txt"

In [4]:
def download_imdb_n(id_list = "imdb_id25K.txt", out_file = "data_imdb_top100.txt", n = 100):
    ##### function downloads top n entires of imdb data as given by imdb_id25K.txt
    ##### outputs downloaded data as 

    # Import TMDb top 25,000 movies using imdb_ids
    ID = np.array(pd.read_csv(id_list, header=None))
    ID = ID[:n]

    # Among the different variables, we select the following variables of interest in our analysis  
    columns = ['title','genres', 'director', 'distributors', 'year', 'rating', 'votes', 'runtimes', 
              'language codes', 'languages', 'producer', 'mpaa', 'writer', 'top 250 rank', 'kind', 
               'country codes', 'countries', 'cover url', 'aspect_ratio', 'production companies', 
               'cinematographer', 'plot outline', 'plot', 'cast', 'animation department', 'original music',
               'canonical title', 'editorial department', 'canonical title', 'long imdb title',
               'long imdb canonical title', 'smart canonical title', 'smart long imdb canonical title',
               'full-size cover url']

    ## Download IMDb data
    index = range(1, len(ID)+1) 
    df = pd.DataFrame(index = index, columns=columns)
    df = df.fillna(0)

    # Fill in dataframe df 
    from imdb import IMDb
    ia = IMDb()
    for i in range(0, len(index)):
        movie = ia.get_movie(ID[i]) # grab movie data by id
        ia.update(movie)
        keys = movie.keys() # generate the available keys of this particular movie 
        print i

        for j in range(0, len(columns)):
            if columns[j] in keys:
                if type(movie[columns[j]]) == list:
                    result = str(movie[columns[j]])
                else:
                    result = movie[columns[j]]
                df.iloc[i,j] = result
            else:
                df.iloc[i,j] = 'nan'

    # add column if IDs to the dataframe
    df['imdb_ids'] = pd.Series(ID.reshape(n,), index = df.index)

    # export dataframe to txt file
    df.to_csv(out_file, encoding='utf-8')

In [ ]:
download_imdb_n(id_list = "imdb_id25K.txt", out_file = "data_imdb_top100.txt", n = 100)

---
## Part2. Parse and Merge Data

For demenstration of code:
use the the top 100 rows

### 2.1 Parse Data

by Zecai

input:

data file "data_imdb_top100.txt"


output:

data file "data_imdb_top100_parse.txt"

### 2.3 Merge TMDB and IMDB

descripe how we merge features and genres

by Wenshan


input:

data file "data_tmdb_top100.txt"

data file "data_tmdb_top100_parse.txt"


output: 

data file "data_features_top100.txt"
    
data file "data_genres_top100.txt"

---
## Part3. Feature Engineering

For demenstration of code:
use variable `director` for the top 100 data

### 2.1 Variables regarding people/companies involved production (DONE)

by Danqing

In [7]:
#val_group1 = ["director", "writer","cast","distributors", "producer", "production companies","cinematographer", 
#              "animation department", "original music", "editorial department"]

#### 2.1.1 extract important top X IDs (for each genre)


#### Directors

In [11]:
## Function that takes in features csv and genres csv 
## and outputs list of top 5 directors associated with each genre

def directors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt"):
    import numpy as np 
    import pandas as pd
    import re
    import seaborn as sns 
    
    features = pd.read_csv(features_csv)
    genres = pd.read_csv(genre_csv)
    
    features2 = features
    features2["mpaa_reason"] = features["mpaa"]

    # parse director column into entries instead of str
    for i in range(len(features)):
        for val in ["director"]:
            if type(features.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                st = features.ix[i,val]
                value = re.findall(r'\d+',st)
                features2[val][i] = value
   
    director = features2['director']
        
    ##### list of directors simply by expanding out each of the 100 movie entries 
    director_list = []
    for i in range(len(director)):
        if director[i] > 0:
            for j in range(len(director[i])):
                director_list = np.append(director_list, director[i][j]) #list of directors (not unique)
    
    ##### dataframe of directors as index, genres as columns 
    director_genre = pd.DataFrame(index = director_list, columns = genres.columns)
    count = 0

    for i in range(len(director)):
        if director[i] > 0:
            for j in range(len(director[i])):
                director_genre.ix[count,] = genres.ix[i,]
                count = count + 1
    
    ##### dataframe of unique directors as index, genre as columns, counts for each genre as table content
    director_full = pd.DataFrame(index = np.unique(director_list), columns = genres.columns)

    for i in range(len(np.unique(director_list))):
        director_full.ix[i, ] = director_genre[director_genre.index == np.unique(director_list)[i]].sum(axis = 0)
    
    #### Clean dataframe
    director_full = director_full.drop('Unnamed: 0', 1) #remove first column 
    total_numbers_by_genre = director_genre.drop('Unnamed: 0', 1).sum(axis =0) #total number of each genre
    percentage = director_full/total_numbers_by_genre #director_full normalized by genre 
    
    #### Select the top five directors of each genre
    genre_director=pd.DataFrame(index = percentage.columns, columns = [1,2,3,4,5])
    for i in range(len(percentage.columns)):
        if percentage[percentage.columns[i]].sum() > 0:
            for j in range(0,5):
                genre_director.iloc[i,j] = percentage.sort(columns = percentage.columns[i], ascending=False).index.values[j]
    
    return(genre_director)

#### Distributors

In [12]:
## Function that takes in features csv and genres csv 
## and outputs list of top 5 distributors associated with each genre

def distributors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt"):
    import numpy as np 
    import pandas as pd
    import re
    import seaborn as sns 
    
    features = pd.read_csv(features_csv)
    genres = pd.read_csv(genre_csv)
    
    features2 = features
    features2["mpaa_reason"] = features["mpaa"]

    # parse director column into entries instead of str
    for i in range(len(features)):
        for val in ["distributors"]:
            if type(features.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                st = features.ix[i,val]
                value = re.findall(r'\d+',st)
                features2[val][i] = value
   
    distributors = features2['distributors']
        
    ##### list of distributors simply by expanding out each of the 100 movie entries 
    distributors_list = []
    for i in range(len(distributors)):
        if distributors[i] > 0:
            for j in range(len(distributors[i])):
                distributors_list = np.append(distributors_list, distributors[i][j]) #list of distributors (not unique)
    
    ##### dataframe of directors as index, genres as columns 
    distributors_genre = pd.DataFrame(index = distributors_list, columns = genres.columns)
    count = 0
    for i in range(len(distributors)):
        if distributors[i] > 0:
            for j in range(len(distributors[i])):
                distributors_genre.ix[count,] = genres.ix[i,]
                count = count + 1
    
    ##### dataframe of unique directors as index, genre as columns, counts for each genre as table content
    distributors_full = pd.DataFrame(index = np.unique(distributors_list), columns = genres.columns)
    for i in range(len(np.unique(distributors_list))):
        distributors_full.ix[i, ] = distributors_genre[distributors_genre.index == np.unique(distributors_list)[i]].sum(axis = 0)
    
    #### Clean dataframe
    distributors_full = distributors_full.drop('Unnamed: 0', 1) #remove first column 
    total_numbers_by_genre = distributors_genre.drop('Unnamed: 0', 1).sum(axis =0) #total number of each genre
    percentage = distributors_full/total_numbers_by_genre #director_full normalized by genre 
    
    #### Select the top five directors of each genre
    genre_distributors=pd.DataFrame(index = percentage.columns, columns = [1,2,3,4,5])
    for i in range(len(percentage.columns)):
        if percentage[percentage.columns[i]].sum() > 0:
            for j in range(0,5):
                genre_distributors.iloc[i,j] = percentage.sort(columns = percentage.columns[i], ascending=False).index.values[j]
    
    return(genre_distributors)

#### Cast

In [13]:
## Function that takes in features csv and genres csv 
## and outputs list of top 5 cast members associated with each genre

def cast_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt"):
    import numpy as np 
    import pandas as pd
    import re
    import seaborn as sns 
    
    features = pd.read_csv(features_csv)
    genres = pd.read_csv(genre_csv)
    
    features2 = features
    features2["mpaa_reason"] = features["mpaa"]

    # parse director column into entries instead of str
    for i in range(len(features)):
        for val in ["cast"]:
            if type(features.ix[i,val]) == str:  # when the entry is not 'float(nan)'
                st = features.ix[i,val]
                value = re.findall(r'\d+',st)
                features2[val][i] = value
   
    cast = features2['cast']
        
    ##### list of distributors simply by expanding out each of the 100 movie entries 
    cast_list = []
    for i in range(len(cast)):
        if cast[i] > 0:
            for j in range(len(cast[i])):
                cast_list = np.append(cast_list, cast[i][j]) #list of distributors (not unique)
    
    ##### dataframe of directors as index, genres as columns 
    cast_genre = pd.DataFrame(index = cast_list, columns = genres.columns)
    count = 0
    for i in range(len(cast)):
        if cast[i] > 0:
            for j in range(len(cast[i])):
                cast_genre.ix[count,] = genres.ix[i,]
                count = count + 1
    
    ##### dataframe of unique directors as index, genre as columns, counts for each genre as table content
    cast_full = pd.DataFrame(index = np.unique(cast_list), columns = genres.columns)
    for i in range(len(np.unique(cast_list))):
        cast_full.ix[i, ] = cast_genre[cast_genre.index == np.unique(cast_list)[i]].sum(axis = 0)
    
    #### Clean dataframe
    cast_full = cast_full.drop('Unnamed: 0', 1) #remove first column 
    total_numbers_by_genre = cast_genre.drop('Unnamed: 0', 1).sum(axis =0) #total number of each genre
    percentage = cast_full/total_numbers_by_genre #director_full normalized by genre 
    
    #### Select the top five directors of each genre
    genre_cast=pd.DataFrame(index = percentage.columns, columns = [1,2,3,4,5])
    for i in range(len(percentage.columns)):
        if percentage[percentage.columns[i]].sum() > 0:
            for j in range(0,5):
                genre_cast.iloc[i,j] = percentage.sort(columns = percentage.columns[i], ascending=False).index.values[j]
    
    return(genre_cast)

In [14]:
directors_top5 = directors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt")
distributors_top5 = distributors_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt")
cast_top5 = cast_by_genre(features_csv ="feature_multi_top100.txt", genre_csv = "genre_multi_top100.txt")

#### 2.1.2 Dimention Reduction Analysis


output: 

a data frame from column `director` that contains that first i PCs and each column is names "director_pci"

### 2.2 Variables for text analysis

by Zecai

In [ ]:
# val_group2 = ["title","plot outline", "plot", "mpaa_reason", "tagline_TMDB"]

In [ ]:
#### 2.2.1 bag-of-words

In [ ]:
#### 2.2.2 dimention reduction

output: 

a data frame from column `title` that contains that first i PCs and each column is names "title_pci"

## Other discusions to include

The exact form of X and Y depends on the ideas you had previously. In general though Y should involve the genre of a movie, and X the features you want to include to predict the genre. Remember from the lecture that more features does not necessarily equal better prediction performance. Use your application knowledge and the insight you gathered from your genre pair analysis and additional EDA to design Y. Do you want to include all genres? Are there genres that you assume to be easier to separate than others? Are there genres that could be grouped together? There is no one right answer here. We are looking for your insight, so be sure to describe your decision process in your notebook.

In preparation for the deep learning part we strongly encourage you to have two sets of training data X, one with the metadata and one with the movie posters. Make sure to have a common key, like the movie ID, to be able to link the two sets together. Also be mindful of the data rate when you obtain the posters. Time your requests and choose which poster resolution you need. In most cases w500 should be sufficient, and probably a lower resolution will be fine.

The notebook to submit this week should at least include:

Discussion about the imbalanced nature of the data and how you want to address it

Description of your data

What does your choice of Y look like?

Which features do you choose for X and why?

How do you sample your data, how many samples, and why?

Important: You do not need to upload the data itself to Canvas.